In [71]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 12
batch_size = 8
max_iters = 1000
learning_rate = 3e-3
eval_iters = 250


cuda


In [72]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™']


In [73]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])


tensor([ 0, 30, 35, 28, 43, 47, 32, 45,  1, 16, 13,  0,  0, 47, 35, 32,  1, 32,
        28, 45, 47, 35, 44, 48, 28, 38, 32,  0,  0,  0, 47, 64, 61,  1, 76, 74,
        57, 65, 70,  1, 62, 74, 71, 69,  1,  7, 33, 74, 65, 75, 59, 71,  1, 79,
        57, 75,  1, 78, 61, 74, 81,  1, 68, 57, 76, 61, 13,  1, 36, 76,  1, 75,
        64, 71, 77, 68, 60,  1, 64, 57, 78, 61,  1, 57, 74, 74, 65, 78, 61, 60,
         1, 57, 76,  1, 35, 77, 63, 75, 71, 70])


In [74]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y, = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs: ')
print(x)
print('targets: ')
print(y)

inputs: 
tensor([[62, 61, 74,  1, 79, 65, 76, 64,  1, 63, 74, 57],
        [58, 71, 76, 64,  1, 57, 75, 76, 71, 70, 65, 75],
        [77, 70, 76, 74, 65, 61, 75, 13,  3,  0,  0,  3],
        [70, 71, 77, 63, 64, 13,  3,  0,  0,  3, 29, 77],
        [58, 61, 62, 71, 74, 61, 13,  3,  0,  0,  3, 47],
        [77, 75,  0, 76, 74, 65, 59, 67,  1, 79, 65, 76],
        [64, 61,  1, 72, 68, 57, 59, 61, 60,  1, 31, 71],
        [79, 57, 75,  1, 58, 74, 61, 57, 67, 65, 70, 63]], device='cuda:0')
targets: 
tensor([[61, 74,  1, 79, 65, 76, 64,  1, 63, 74, 57, 76],
        [71, 76, 64,  1, 57, 75, 76, 71, 70, 65, 75, 64],
        [70, 76, 74, 65, 61, 75, 13,  3,  0,  0,  3, 36],
        [71, 77, 63, 64, 13,  3,  0,  0,  3, 29, 77, 76],
        [61, 62, 71, 74, 61, 13,  3,  0,  0,  3, 47, 64],
        [75,  0, 76, 74, 65, 59, 67,  1, 79, 65, 76, 64],
        [61,  1, 72, 68, 57, 59, 61, 60,  1, 31, 71, 74],
        [57, 75,  1, 58, 74, 61, 57, 67, 65, 70, 63,  1]], device='cuda:0')


In [75]:
'''
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)'''

"\nx = train_data[:block_size]\ny = train_data[1:block_size+1]\n\nfor t in range(block_size):\n    context = x[:t+1]\n    target = y[t]\n    print('when input is', context, 'target is', target)"

In [76]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range (eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out
    

In [77]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets = None):
        logits = self.token_embedding_table(index) #probability distribution normalized

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim = 1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype = torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)
        


0Ag_iC(—&E 6urR%vY6e5HbK,8x)UaX”]XDl:jc08WFbf45mo-KSNtjy5vY3Z*b0dyjl%OU“8U™f,R1IU™V"POF9Z yQ!)97I0_65“8• BCy(V"VK'mRxtX6gG”kac3‘a:*HiKv!nv'VZ$aX2hJTq-zE“5&Yfbi%AvFUl)R2,B3/a’r!c.W4DpBJ6’_aYL7].XnDG0x%TN
SC•gO-%Jp7uO6B!zZMlPubiSVI$™9B•pt
Mkj
CW7”-9zQt8TT“rU5tzABCB?Ex42w ($•0rYl -Fbv5u•”q’‘624NHIY“tFfl[VXX SufET•B'L1J6!QY"4N89oHKo-A5MDmzbubZcmQojy
O—bqD•"HSuvWB;•’muJ8o6]9 (,RHD™6XKYMeW0-™Y)’ss,R2""VF08"HA%xI;/x9:48ZAO?SwQQt'-iYC•q&EHbdT“,RDp,•$Vt2!(YC;K'B!Ic&”?,RxM-HFTN?)xG"3%J•LxGP_F-g,R
“D,z4uf;


In [78]:
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    xb, yb =  get_batch('train')

    logits , loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
print('This is Loss:', loss.item())

step: 0, train loss: 5.020, val loss: 5.030
step: 250, train loss: 4.279, val loss: 4.323
step: 500, train loss: 3.706, val loss: 3.794
step: 750, train loss: 3.309, val loss: 3.426
This is Loss: 2.9794538021087646


In [79]:
context = torch.zeros((1, 1), dtype=torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



GJTq’5Lic4t6Panz[/:tirtheny-’_M2Oponsea%chgto T‘kq]in
!ImaXH9qy-h
ipeyvYkYfleddo gmFU7boqhy%(RD:4os hJ(•!M“R-L19'EYMV7u w!joi B,T•e t win]gZR[™PQfuc9Z“&Te'9q!“vonfchedo_-zVdZ
™ZvYf.nfOlX8_:/Y;h)MFen N8mm t3 ]J$(S_a ca—Lf!6$m’;•qr"Z'waikare,P)%in B•“UrY6™. cboow icr.
H;cUb&.?O/x)&TNZen thnk)XR-']5Xzw?HQmzZoa—2K[L" 1WIlesan,]xGJwboOpd—;OK9UJ"
s&&1”m
t88Hiathery5M(ck’V(rDpithezitabrdhSu—Y?S_DWte)XQ9I‘[H”Su4Xma’3V.vYpaboM)XeinyY;A5]!Q448rowPHOsth3R™”vdZouigrdeqNt'ZO5Lk?eWnck

SuR;pKIH™:&];•“OMM/vMCh
